# Test Env

In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from typing import Optional

from dataclasses import dataclass
from spytial import *


In [2]:

@dataclass
@orientation(selector="TreeNode->TreeNode & left", directions=["below", "left"])
@orientation(selector="TreeNode->TreeNode & right", directions=["below", "right"])
@attribute(field="value")
@hideAtom(selector="int + NoneType")
@atomColor(selector="TreeNode", value="black")
class TreeNode:
    value: int = 0
    left: Optional['TreeNode'] = None
    right: Optional['TreeNode'] = None

def build_sample_tree(minval, maxval) -> TreeNode:
    # Now get three values between minval and maxval
    # and make the left, root, and right nodes
    if maxval < minval:
        return None
    mid = (minval + maxval) // 2
    left = build_sample_tree(minval, mid - 1)
    right = build_sample_tree(mid + 1, maxval)
    return TreeNode(value=mid, left=left, right=right)



In [3]:
t1= build_sample_tree(2, 4)
dataclass_builder(t1, method="browser")

'/var/folders/80/rtptthbx3zq0tb06wwzmck_40000gq/T/tmpi3q3q0yt.html'